In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 27 11:45:24 2019

@author: Salar
"""

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset


class t_net(nn.Module):
    def __init__(self):
        super(t_net, self).__init__()
        self.dense1 = nn.Linear(2, 2, True)
        self.dense2 = nn.Linear(2,2, True)
        self.bn = nn.BatchNorm1d(2)
        
    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        #x = self.bn(x)
        return x
    

class r_net(nn.Module):
    def __init__(self):
        super(r_net, self).__init__()
        self.dense1 = nn.Linear(2, 2, True)
        self.dense2 = nn.Linear(2,2, True)
        
    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        x = F.softmax(x)
        return x


Learning_Rate = 0.0001
SNR_dB = [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
SNR = np.power(10,(np.matrix(SNR_dB)/10))

data_size = 10000
temp = np.random.randint(2, size = data_size)
noise = np.random.rand(data_size)


Alice = t_net()
Bob = r_net()
Eve = r_net()

criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.CrossEntropyLoss()

Alice_optimizer = torch.optim.SGD(Alice.parameters(), lr = Learning_Rate)
Bob_optimizer = torch.optim.SGD(Bob.parameters(), lr = Learning_Rate)
Eve_optimizer = torch.optim.SGD(Eve.parameters(), lr = Learning_Rate)


Alice_optimizer.zero_grad()
Eve_optimizer.zero_grad()
Bob_optimizer.zero_grad()


Bob_Lab = np.zeros((np.size(SNR),data_size))        
Eve_Lab = np.zeros((np.size(SNR),data_size))


for j in range (np.size(SNR)):
    
    
    Signal_Bob = (np.sqrt(SNR[0,j]/2)) * temp + np.sqrt(1/2) * noise
    Signal_Eve = (np.sqrt(SNR[0,j]/2)) * temp + np.sqrt(1/2) * noise + 0.3 * noise

    for i in range (data_size):
        if(Signal_Bob [i] > 0.5):
            images_Bob = torch.FloatTensor([1.0, 0.0]).reshape(1,2)
            labels_Bob = torch.tensor([1])
        else:
            images_Bob = torch.FloatTensor([0.0, 1.0]).reshape(1,2)
            labels_Bob = torch.tensor([0])
        
        if(Signal_Eve [i] > 0.5):
            images_Eve = torch.FloatTensor([1.0, 0.0]).reshape(1,2)
            labels_Eve = torch.tensor([1])
        else:
            images_Eve = torch.FloatTensor([0.0, 1.0]).reshape(1,2)
            labels_Eve = torch.tensor([0])
        if(temp [i] > 0.5):
            images_Alice = torch.FloatTensor([1.0, 0.0]).reshape(1,2)
            labels_Alice = torch.tensor([1])
        else:
            images_Alice = torch.FloatTensor([0.0, 1.0]).reshape(1,2)
            labels_Alice = torch.tensor([0])
    
            
        Alice_optimizer.zero_grad()
        Eve_optimizer.zero_grad()
        Bob_optimizer.zero_grad()
        
        Bob_Out = Bob(images_Bob)
        Eve_Out = Eve(images_Eve)
        Alice_Out = Alice(images_Alice)
        
        Bob_Lab[j,i] = np.argmax((Bob_Out.detach().numpy()))
        Eve_Lab[j,i] = np.argmax((Eve_Out.detach().numpy()))
        
        
        Loss = criterion3(Alice_Out, labels_Alice)
        
        Loss_1 = criterion1(Bob_Out, labels_Bob)
        
        Loss_2 = 1 * criterion2(Eve_Out, labels_Eve)

        
        print(Loss_1 - Loss_2)

        Loss.backward()
        
        Alice_optimizer.step()
        Bob_optimizer.step()
        Eve_optimizer.step()
        
        Loss_1.backward()
        
        Alice_optimizer.step()
        Bob_optimizer.step()
        Eve_optimizer.step()
        
        Loss_2.backward()
        
        Alice_optimizer.step()
        Bob_optimizer.step()
        Eve_optimizer.step()
    
    
Bob_Error = np.zeros((np.size(SNR)))
Eve_Error = np.zeros((np.size(SNR)))

for j in range(np.size(SNR)):
    for i in range(data_size):
        if(Bob_Lab[j,i] != temp[i]):
            Bob_Error[j] = Bob_Error[j] + 1
        if(Eve_Lab[j,i] != temp[i]):
            Eve_Error[j] = Eve_Error[j] + 1


Bob_BER = Bob_Error/data_size
Eve_BER = Eve_Error/data_size

plt.semilogy(np.asarray(SNR_dB),Bob_BER)
plt.xlabel('SNR (dB)')
plt.ylabel('Bit Error Rate')
plt.xlim(-3,15)



plt.semilogy(np.asarray(SNR_dB),Eve_BER)
plt.xlabel('SNR (dB)')
plt.ylabel('Bit Error Rate')
plt.xlim(-3,15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.2341, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.2341, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.1150, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.2341, grad_fn=<SubBackward0>)
tensor(-0.2341, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.1150, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.1148, grad_fn=<SubBackward0>)
tensor(-0.2341, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.1147, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(0.2839, grad_fn=<SubBackward0>)
tensor(-0.2341, 

In [ ]:
print(Bob_Lab)
print(Eve_Lab)
print(temp)
print(Bob_Lab - temp)
print(Eve_Lab - temp)

[[1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[0 0 1 0 0 1 1 0 0 1]
[[1. 0. 0. 1. 1. 0. 0. 1. 1. 0.]
 [1. 1. 0. 1. 1. 0. 0. 1. 1. 0.]
 [1. 1. 0. 1. 1. 0. 0. 1. 1. 0.]
 [1. 1. 0. 1. 1. 0.